We begin by considering $ K_4 $ as an example, and compute the minimal spanning tree T. Set S will be all edges not in T. Weights will not be considered.

In [43]:
#G = Graph([[1,2],[2,3],[3,1],[3,4]])
G = graphs.CompleteGraph(4)
T_weighted = G.min_spanning_tree()
T = [(x,y) for (x,y,z) in T_weighted]
S = [x for x in G.edges(labels=False) if not(x in T) and not((x[1],x[0]) in T)]

For each $ e \in S $ and all $ f \in T $, we determine the cycle c resulting from adding $ e $ to $ T $, and find $$
c_{ef}^T =
\begin{cases} 
1 & \text{if } f = [u,v] \text{ is in } c \\ 
-1 & \text{if } -f = [v,u] \text{ is in } c \\ 
0 & \text{if } f \text{ is not in } c
\end{cases}
$$

We will then compute $ C_t = [c_{ef}^T]_{e \in S,\ f \in T} $.

  

In [86]:
C_t = []
MtXt = []

nonzero_entries = 0

m = {}

for s in S:
    _T = Graph(T + [s])
    c = [ c for c in _T.to_directed().all_simple_cycles() if c[1]<c[-2]][0]
    c_edges = [(c[i],c[i+1]) for i in range(len(c) - 1)]
    _s = []
    _m = []
    m[s[0],s[1]] = var(f"m_{s[0]}{s[1]}")
    for t in T:
        if t in c_edges:
            _s.append(1)
            nonzero_entries += 1
        elif (t[1],t[0]) in c_edges:
            _s.append(-1)
            nonzero_entries += 1
        else:
            _s.append(0)
        m[t[0],t[1]] = var(f"m_{t[0]}{t[1]}")
        _m.append(m[s[0],s[1]] - m[t[0],t[1]])
        
    C_t.append(_s)
    MtXt.append(_m)

[NEED TO VERIFY] We then multiply $ C_t \begin{bmatrix}
x_2 - x_1 \\
x_3 - x_1 \\
x_4 - x_1
\end{bmatrix}  $ and get $ c_{ef}^T \mapsto c_{ef}^T (m_e - m_f) $ for each entry of the matrix.

In [84]:
M = [[c * m for c, m in zip(row_c, row_m)]
          for row_c, row_m in zip(C_t, MtXt)]

We take the determinant of the resulting matrix for the slope circuit polynomial $ p $. We now will establish a polynomial ring in variables $ m_ik $ and compute some basic properties of $ p $.

In [87]:
slopes = sorted(str(v) for v in m.values())

R = PolynomialRing(QQ, slopes)

p = R(Matrix(M).det())

print(f'Polynomial:  p(x) = {p}')
print(f'Degree: {p.degree()}')
print(f'# of terms: {p.number_of_terms()}')
print(f'# of nonzero matrix entires: {nonzero_entries}')

Polynomial:  p(x) = -m_10*m_13*m_20 + m_12*m_13*m_20 - m_10*m_12*m_23 + m_10*m_13*m_23 + m_10*m_20*m_23 - m_13*m_20*m_23 + m_10*m_12*m_30 - m_12*m_13*m_30 - m_12*m_20*m_30 + m_13*m_20*m_30 - m_10*m_23*m_30 + m_12*m_23*m_30
Degree: 3
# of terms: 12
# of nonzero matrix entires: 6
